# 전체 크롤링 시작

In [1]:
import requests
from bs4 import BeautifulSoup 
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# category별 데이터 저장소 생성. {레스토랑&카페 : {'카페/디저트': [],...}} 이형태로 저장할거.
cate_total_data = {}

path = "C:\\Users\\A\\Asia_AI\\chromedriver"
driver = webdriver.Chrome(path) ## 경로 설정
driver.implicitly_wait(3)

base_url = 'https://www.starfield.co.kr/coexmall/tenant/categoryInfo.do'
driver.get(base_url)

all_xpath = '//*[@id="cateOneList"]'
sub_all_xpath = '//*[@id="cateTwoText"]'

# 큰 카테고리 XPath 자동으로 저장하기
big_cate_xpath = {} # {'레스토랑&카페': '//*[@id= "CT201802061015172099"]',..}  이런형태
cate_xpath = {} # {'레스토랑&카페':{'CJ푸드몰':'//*[@id= "CT202.."]','카페/디저트': '//*[@id= "CT201802061051412248"]',..}

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

find_ul = soup.find('ul', 'selectList')
find_li = find_ul.find_all('a')
for id_tag in  find_li[1:]:
    big_cate_xpath[id_tag.text] = '//*[@id= ' + '"' + id_tag.get('id') + '"]'

# 큰 카테고리별 : 작은 카테고리 xpath 전부 추가하기
for big_cate, xpath in big_cate_xpath.items():
    driver.find_element_by_xpath(all_xpath).send_keys(Keys.ENTER)
    driver.implicitly_wait(3)
    driver.find_element_by_xpath(xpath).click()
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    find_ul = soup.find_all('ul', 'selectList')[1]
    find_li = find_ul.find_all('a')
    temp = {}
    for id_tag in  find_li[1:]:
        temp[id_tag.text] = '//*[@id= ' + '"' + id_tag.get('id') + '"]'
    
    cate_xpath[big_cate] = temp

# big_cate for문 안에 small_cate for문 돌려서 차례대로 크롤링.
for big_cate, small_cate in cate_xpath.items():
#     if big_cate in ['레스토랑&카페', '패션의류', '패션잡화', '뷰티', '라이프스타일']:
#         continue
    print('카테고리:' ,big_cate)
    driver.find_element_by_xpath(all_xpath).send_keys(Keys.ENTER)
    driver.implicitly_wait(3) # 셀레니움 속도가 느리기 때문에 실행되기 까지 최대 3초 기다려주기
    driver.find_element_by_xpath(big_cate_xpath[big_cate]).click()
    
    temp = {}
    for cate, xpath in small_cate.items():
        print('\t\t' ,cate)
        driver.find_element_by_xpath(sub_all_xpath).send_keys(Keys.ENTER)
        driver.implicitly_wait(3)
        driver.find_element_by_xpath(xpath).click()
        
        # 더보기 최대 10번 클릭 시도
        for i in range(10) :
            try :
                driver.find_element_by_xpath('//*[@id="viewMore"]/button').click()
            except :
                pass
    
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        find_ul = soup.find('ul', 'store-list')
        find_li = find_ul.find_all('li', 'cube')
        
        temp[cate] = []
        for tag in find_li:
            store_name = tag.find('span', class_='brand-title-ko').text
            inform_tag = tag.find_all('div', class_='info-wrap')
            store_call = inform_tag[1].find('span', class_='ko').text
            store_floor = inform_tag[2].find('span', class_='ko').text

            temp[cate].append([store_name,store_floor,store_call])
            
    cate_total_data[big_cate] = temp
    
print("cate_total_data 생성 완료")
driver.close()

카테고리: 레스토랑&카페
		 CJ푸드몰
		 카페/디저트
		 한식
		 양식
		 아시안
		 중식
		 일식
		 퓨전
		 분식
		 패스트푸드
카테고리: 패션의류
		 여성의류
		 남성의류
		 SPA
		 스트리트/캐주얼
		 유니섹스
		 의류멀티샵
		 언더웨어/양말
		 스포츠
		 아웃도어
카테고리: 패션잡화
		 슈즈/백
		 액세서리/모자
		 안경/선글라스
		 주얼리/시계
카테고리: 뷰티
		 코스메틱
		 바디&향수
카테고리: 라이프스타일
		 자동차/바이크/사이클
		 가전/휴대폰
		 도서/음반
		 디자인용품/기프트
		 문구/기프트
		 여행구
		 취미용품
		 화원
카테고리: 홈퍼니싱
		 가구/생활용품
		 리빙퍼퓸/향초
카테고리: 키즈
		 키즈의류
		 완구
		 키즈까페
카테고리: 엔터테인먼트
		 극장
		 아쿠아리움
		 게임센터
카테고리: 슈퍼마켓
		 슈퍼마켓
카테고리: 서비스
		 뷰티케어서비스
		 생활서비스
		 헬스케어서비스
cate_total_data 생성 완료


In [10]:
# 전체 상점 데이터 카테고리별 속성 네임으로 txt 파일 생성.
file_name_list = []
for big_cate, small_group in cate_total_data.items():
    for small_cate, store_infos in small_group.items():
        # 상점내용 없으면 파일생성 X
        if len(store_infos) == 0:
            continue
        name = big_cate+'_'+small_cate+'.txt'
        file_name = name.replace('/', '와 ')
        action = "wt"
        
        if small_cate in ['유니섹스', '의류멀티샵']:
            file_name = '패션의류_스트리트와 캐주얼.txt'
            action = "at"
            
        elif small_cate == '주얼리/시계':
            file_name = '패션잡화_액세서리와 모자.txt'
            action = "at"
            
        elif small_cate == '코스메틱':
            file_name='뷰티_화장품.txt'
            
        elif small_cate == '게임센터':
            file_name='엔터테인먼트_게임장.txt'            
            
        elif small_cate == '뷰티케어서비스':
            file_name='서비스_미용샵.txt'     
            
        with open(file_name, "wt") as f:
            for info in store_infos:
                fixed_info = info[0]+'<br>'+'위치: '+info[1]+'층'+'<br>'+"대표 전화: "+info[2]
                f.write(fixed_info+'\n')
        
        if action == "at":
            continue
            
        file_name_list.append(file_name)
        print('{} 파일 생성 완료'.format(file_name))

레스토랑&카페_CJ푸드몰.txt 파일 생성 완료
레스토랑&카페_카페와 디저트.txt 파일 생성 완료
레스토랑&카페_한식.txt 파일 생성 완료
레스토랑&카페_양식.txt 파일 생성 완료
레스토랑&카페_아시안.txt 파일 생성 완료
레스토랑&카페_중식.txt 파일 생성 완료
레스토랑&카페_일식.txt 파일 생성 완료
레스토랑&카페_퓨전.txt 파일 생성 완료
레스토랑&카페_분식.txt 파일 생성 완료
레스토랑&카페_패스트푸드.txt 파일 생성 완료
패션의류_여성의류.txt 파일 생성 완료
패션의류_남성의류.txt 파일 생성 완료
패션의류_SPA.txt 파일 생성 완료
패션의류_스트리트와 캐주얼.txt 파일 생성 완료
패션의류_언더웨어와 양말.txt 파일 생성 완료
패션의류_스포츠.txt 파일 생성 완료
패션의류_아웃도어.txt 파일 생성 완료
패션잡화_슈즈와 백.txt 파일 생성 완료
패션잡화_액세서리와 모자.txt 파일 생성 완료
패션잡화_안경와 선글라스.txt 파일 생성 완료
뷰티_화장품.txt 파일 생성 완료
뷰티_바디&향수.txt 파일 생성 완료
라이프스타일_가전와 휴대폰.txt 파일 생성 완료
라이프스타일_도서와 음반.txt 파일 생성 완료
라이프스타일_디자인용품와 기프트.txt 파일 생성 완료
라이프스타일_문구와 기프트.txt 파일 생성 완료
라이프스타일_여행구.txt 파일 생성 완료
라이프스타일_취미용품.txt 파일 생성 완료
라이프스타일_화원.txt 파일 생성 완료
홈퍼니싱_가구와 생활용품.txt 파일 생성 완료
홈퍼니싱_리빙퍼퓸와 향초.txt 파일 생성 완료
키즈_키즈의류.txt 파일 생성 완료
엔터테인먼트_극장.txt 파일 생성 완료
엔터테인먼트_아쿠아리움.txt 파일 생성 완료
엔터테인먼트_게임장.txt 파일 생성 완료
슈퍼마켓_슈퍼마켓.txt 파일 생성 완료
서비스_미용샵.txt 파일 생성 완료
서비스_생활서비스.txt 파일 생성 완료
서비스_헬스케어서비스.txt 파일 생성 완료


In [ ]:
##### 주차정보 찾기
path = "C:\\Users\\A\\Asia_AI\\chromedriver"
driver = webdriver.Chrome(path) ## 경로 설정
driver.implicitly_wait(1)

prk_url='https://www.starfield.co.kr/coexmall/about/parkingInfo.do'
driver.get(prk_url)

parking_data = {}

group_prk = driver.find_element_by_class_name('parking-wrap')
div_tags = group_prk.find_elements_by_tag_name('div')

for i,div in enumerate(div_tags):
    
    if i ==0:
        title = div.find_element_by_class_name('transportation').text
        floor_info=div.find_elements_by_class_name('parking-floor')
        temp =[]
        for info in floor_info:
            detail_info = info.text
            temp.append(detail_info)
        parking_data[title] = temp   
    
    if i == 2 :
        title = div.find_element_by_class_name('transportation').text
        details = div.find_elements_by_class_name('parking-detail')
        temp =[]
        for detail in details:
            front_title = detail.find_element_by_class_name('route-name').text
            temp_1 = ""
            for span in detail.find_elements_by_tag_name('span')[1:]:
                temp_1 += span.text + ", "
            
            if detail.find_elements_by_tag_name('li') != None:
                temp_2 = ""
                for li in detail.find_elements_by_tag_name('li'):
                    temp_2 += li.text.replace("\n", '이면 ') + ", "
                temp.append(front_title+":"+temp_1+temp_2)
                continue
                
            temp.append(front_title+":"+temp_1)
        parking_data[title] = temp
    
    if i == 5:
        title = div.find_element_by_class_name('transportation').text
        texts = div.find_element_by_tag_name('dl').text
        parking_data[title] = [texts]
        
driver.close()

for big_cate, small_group in parking_data.items():
        name = big_cate+'.txt'
        with open(name, "wt", encoding='utf-8') as f:
            for info in small_group:
                f.write(info+'\n')
            print('{} 파일 생성 완료'.format(name))

In [38]:
###### 파일 이름으로 불러와서 intents 파일 생성하기 위해 처리해주는 과정.
tag_dict = []
멘트=["어디있어요?", "알려주세요","뭐있어?","말해줘", "찾아줘",
    "찾아주세요", "정보", "정보 알려줘", "가고싶어"
   ]

for file in file_name_list:
    tag_name = file.replace('.txt', '')
    words_list = tag_name.split('_')
    s_words = [word.split(' ') for word in words_list]
    words_split_list=[w.replace('와', '') for w in sum(s_words, [])]
    words_split_list = sum([w.split('&') for w in words_split_list], [])
    
    if words_split_list[0] == '레스토랑':
        words_split_list.pop(1)
        if list(words_split_list[1])[-1] == '식':
            words_split_list.append(words_split_list[1]+'당')
            words_split_list.append(words_split_list[1]+'집')
        elif words_split_list[1] == '패스트푸드':
            words_split_list.append('햄버거집')
        elif words_split_list[1] == '분식':
            words_split_list.append('분식집')
        
    elif words_split_list[0] == '패션의류':
        if words_split_list[1] in ['남성의류','여성의류']:
            l = list(words_split_list[1])
            words_split_list.append(l[0]+l[1]+' 패션몰')
        elif words_split_list[1] == 'SPA':
            words_split_list.append('SPA브랜드')
            words_split_list.append('스파브랜드')
        elif words_split_list[1] == '스트리트':
            words_split_list.append('유니섹스몰')
            words_split_list.append('캐주얼 매장')
            words_split_list.append('편집샵')
        elif words_split_list[1] == '스포츠':
            words_split_list.append('스포츠 브랜드')
            words_split_list.append('스포츠 매장')
        elif words_split_list[1] == '아웃도어':
            words_split_list.append('아웃도어 브랜드')
            words_split_list.append('아웃도어 매장')
    
    elif words_split_list[0] == '패션잡화':
        if words_split_list[1] == '슈즈':
            words_split_list.append('신발')
            words_split_list.append('가방')
            words_split_list.append('신발매장')
            words_split_list.append('가방매장')  
        elif words_split_list[1] == '액세서리':
            words_split_list.append('쥬얼리')
            words_split_list.append('시계매장')
            words_split_list.append('목걸이')
            words_split_list.append('반지')
            words_split_list.append('반지매장')
        elif words_split_list[1] == '액세서리':
            words_split_list.append('선글라스매장')
            
            
    elif words_split_list[0] == '뷰티':
        if words_split_list[1] =='화장품':
            words_split_list.append('화장품매장')
            words_split_list.append('화장품가게')
            words_split_list.append('화장품 로드샵')
        elif words_split_list[1] =='바디':
            words_split_list[1] = '바디용품점'
            words_split_list.append('입욕제품 매장')
            words_split_list.append('샤워용품 매장')
    
    elif words_split_list[0] == '라이프스타일':
        if words_split_list[-1] =='휴대폰':
            words_split_list.append('폰')
            words_split_list.append('핸드폰')
            words_split_list.append('전자제품')
            words_split_list.append('모바일')
    
    elif words_split_list[0] == '라이프스타일':
        if words_split_list[1] =='도서':
            words_split_list[1] = '책'
            words_split_list[2] = '앨범'
            words_split_list.append('영풍문고')
        elif words_split_list[1] =='여행구':
            words_split_list[1] = '여행사'
        elif words_split_list[1] =='화원':
            words_split_list[1] = '꽃집'
    
    elif words_split_list[0] == '홈퍼니싱':
        if words_split_list[1] =='가구':
            words_split_list.append('가구점')
            words_split_list.append('생활용품점')
        elif words_split_list[1] =='리빙퍼퓸':
            words_split_list[1] = '디퓨저매장'
    
    elif words_split_list[0] == '키즈':
        words_split_list[1] ='애기옷 매장'
        words_split_list.append('애기의류 매장')
    
    elif words_split_list[0] == '엔터테인먼트':
        if words_split_list[1] =='극장':
            words_split_list[1] = '영화관'
        
    elif words_split_list[0] == '서비스':
        if words_split_list[1] =='미용샵':
            words_split_list.append('미용실')
            words_split_list.append('네일샵')
            words_split_list.append('헤어샵')
        elif words_split_list[1] =='헬스케어서비스':
            words_split_list[1] = '병원'
            words_split_list.append('약국')
            
    patterns_list = []
    for word in words_split_list[1:]:
        patterns_list.append(word)
        for ments in 멘트:
            patterns_list.append(word+' '+ments)

    dict_1 = {}
    dict_1['tag'] = words_list[-1]
    dict_1['patterns'] = patterns_list
    
    with open(file, "rt") as f:
        infos = f.readlines()

        dict_1['responses'] = infos
    
    tag_dict.append(dict_1)
r_intents = {}
r_intents['intents'] = tag_dict

In [39]:
import json
file_path = './intents_starfield_store.json'

with open(file_path, 'w', encoding='utf-8') as json_data:
    json.dump(r_intents, json_data, ensure_ascii=False, indent="\t")

In [25]:
intro_intents

[{'tag': 'greeting',
  'patterns': ['안녕',
   '안녕하세요',
   '잘 지냈어요?',
   '여기요',
   '잘 계셨어요?',
   '거기 누구 있어요?',
   '계세요',
   '계셔요',
   '좋은 날 입니다'],
  'responses': ['안녕하세요, 방문해 주셔서 감사합니다.',
   '다시 만나서 반가워요.',
   '만나서 반갑습니다',
   '찾아 주셔서 감사합니다',
   '안녕하세요, 어떻게 도와 드릴까요?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['안녕히 계셔요.',
   '그럼 나중에 또 들릴께요',
   '나중에요',
   '다음에',
   '다음번에',
   '장사 잘 하세요',
   '돈 많이 버세요',
   '수고하세요',
   '친절한 서비스에 감사합니다.'],
  'responses': ['나중에 또 오세요, 방문해 주셔서 감사합니다.',
   '좋은 하루 되세요.',
   '안녕히 가세요',
   '감사합니다.',
   '고맙습니다',
   '즐거운 시간 되세요']},
 {'tag': 'thanks',
  'patterns': ['감사', '감사합니다', '고맙습니다', '좋은 서비스 감사합니다'],
  'responses': ['도움이 되서 다행입니다.', '당연히 할 일을 한건데요 뭐..', '저도 고맙습니다.', '감사합니다']},
 {'tag': 'Slang',
  'patterns': ['멍청이', '바보', '야임마', '꺼져', '사라져', '새끼', '닥쳐'],
  'responses': ['좋은 말로 해주세요.',
   '듣기가 거북하네요',
   '앞으로 더 노력 하겠습니다.',
   '좋은 모습으로 만나 뵐 수 있기를 바랍니다.']}]

In [34]:
parking_intents

[{'tag': '주차 요금',
  'patterns': ['주차요금이 어떻게 되요?',
   '주차비 알려주세요',
   '승용차 주차요금 얼마에요?',
   '대형버스 주차요금 얼마에요?',
   '화물차 주차요금 얼마에요?',
   '장애인 주차 요금 얼마에요??',
   '국가유공자 주차요금 얼마에요?',
   '국가 유공자 주차 할인 알려주세요',
   '장애인 주차 할인 알려주세요'],
  'responses': ['승용차 :20분 이내 출차시 무료 / 15분당 1,200원 / 1일 주차시 최고 48,000원<br>화물차(2.5t 이상) :승용차 주차요금의 2배<br>대형버스 :스타필드 코엑스몰 주차장 이용불가(탄천주차장 이용)<br>Grace Time :입차 후 20분 이내 출차 차량 – 주차료 면제<br>입차 후 20분 초과 출차 차량 – 정상요금(20분 포함)<br>무인요금계산기 정산 후 출차시 - Grace Time 20분 인정<br>장애인·국가유공자 :50% 할인']},
 {'tag': '주차 할인',
  'patterns': ['주차 할인되나요?',
   '주차 할인 어떻게 되요?',
   '주차비 얼마 할인되나요?',
   '주차 할인 기준 알려줘요',
   '주차 할인 얼마나 되나요?',
   '주차 할인 적용 알려주세요',
   '주차비 할인 적용 알려주세요'],
  'responses': ['주요매장 적용기준 : 스타필드 코엑스몰 영수증 할인, 주요매장 주차할인, 주차할인권은 모두 중복가능<br>주요매장 할인기준 : 스타필드 코엑스몰 영수증 금액기준<br>일부 서비스/편의 매장은 제외(일부 매장별 자체할인 적용)<br>입차 후 구매금액에 한하며, 출차 시 자동정산 후에는 적용되지 않습니다.<br>일부 매장 별 자체할인 적용<br>5만원 이상 : 1시간 주차요금 무료<br>10만원 이상 : 2시간 주차요금 무료<br>15만원 이상 : 최대 3시간 주차요금 무료(3시간 이후 추가요금 발생)<br>주요매장 주차할인<br><br>메가박스(

In [40]:
copy_intents = r_intents.copy()
for p in parking_intents:
    copy_intents['intents'].append(p)
copy_intents['intents']

[{'tag': 'CJ푸드몰',
  'patterns': ['CJ푸드몰',
   'CJ푸드몰 어디있어요?',
   'CJ푸드몰 알려주세요',
   'CJ푸드몰 뭐있어?',
   'CJ푸드몰 말해줘',
   'CJ푸드몰 찾아줘',
   'CJ푸드몰 찾아주세요',
   'CJ푸드몰 정보',
   'CJ푸드몰 정보 알려줘',
   'CJ푸드몰 가고싶어'],
  'responses': ['[임시휴점] 계절밥상<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
   '계절밥상 소반<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
   '방콕9<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
   '빕스마이픽<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
   '제일제면소<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
   '차이나팩토리 익스프레스<br>위치: B1층<br>대표 전화: 1670-0865\n',
   '투썸플레이스<br>위치: B1층<br>대표 전화: 02-1670-0865\n']},
 {'tag': '카페와 디저트',
  'patterns': ['카페',
   '카페 어디있어요?',
   '카페 알려주세요',
   '카페 뭐있어?',
   '카페 말해줘',
   '카페 찾아줘',
   '카페 찾아주세요',
   '카페 정보',
   '카페 정보 알려줘',
   '카페 가고싶어',
   '디저트',
   '디저트 어디있어요?',
   '디저트 알려주세요',
   '디저트 뭐있어?',
   '디저트 말해줘',
   '디저트 찾아줘',
   '디저트 찾아주세요',
   '디저트 정보',
   '디저트 정보 알려줘',
   '디저트 가고싶어'],
  'responses': ['고디바<br>위치: B1층<br>대표 전화: 02-551-5456\n',
   '공차<br>위치: B1층<br>대표 전화: 02-6002-3090\n',
   '공차<br>위치: 1F층<br

In [44]:
cate_total_list = []
for i in copy_intents['intents']:
    cate_total_list.append(i['tag'])
    
dict_1 = {}
dict_1['tag'] = '카테고리'
dict_1['patterns'] = ['카테고리', '카테고리 찾아줘', '카테고리 알려줘']
dict_1['responses']=cate_total_list

total_append_intents = copy_intents.copy()
total_append_intents['intents'].append(dict_1)
for i in intro_intents:
    total_append_intents['intents'].append(i)
total_append_intents

{'intents': [{'tag': 'CJ푸드몰',
   'patterns': ['CJ푸드몰',
    'CJ푸드몰 어디있어요?',
    'CJ푸드몰 알려주세요',
    'CJ푸드몰 뭐있어?',
    'CJ푸드몰 말해줘',
    'CJ푸드몰 찾아줘',
    'CJ푸드몰 찾아주세요',
    'CJ푸드몰 정보',
    'CJ푸드몰 정보 알려줘',
    'CJ푸드몰 가고싶어'],
   'responses': ['[임시휴점] 계절밥상<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
    '계절밥상 소반<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
    '방콕9<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
    '빕스마이픽<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
    '제일제면소<br>위치: B1층<br>대표 전화: 02-1670-0865\n',
    '차이나팩토리 익스프레스<br>위치: B1층<br>대표 전화: 1670-0865\n',
    '투썸플레이스<br>위치: B1층<br>대표 전화: 02-1670-0865\n']},
  {'tag': '카페와 디저트',
   'patterns': ['카페',
    '카페 어디있어요?',
    '카페 알려주세요',
    '카페 뭐있어?',
    '카페 말해줘',
    '카페 찾아줘',
    '카페 찾아주세요',
    '카페 정보',
    '카페 정보 알려줘',
    '카페 가고싶어',
    '디저트',
    '디저트 어디있어요?',
    '디저트 알려주세요',
    '디저트 뭐있어?',
    '디저트 말해줘',
    '디저트 찾아줘',
    '디저트 찾아주세요',
    '디저트 정보',
    '디저트 정보 알려줘',
    '디저트 가고싶어'],
   'responses': ['고디바<br>위치: B1층<br>대표 전화: 02-551-5456\n',
    '공차<br>위치:

In [45]:
import json
file_path = './intents_final.json'

with open(file_path, 'w', encoding='utf-8') as json_data:
    json.dump(total_append_intents, json_data, ensure_ascii=False, indent="\t")

In [349]:
#### 매장별 tag 전체 만들기.
tag_dicts = []
total_store_list = []
멘트=["어디있어요?", "알려주세요", "매장", "찾아주세요", "정보", "가고싶어", "전화번호"]
num = 0
for bit_cate, small_cate in cate_total_data.items():
    for cate, stores in small_cate.items():
        for store in stores:
            if store[0] == '[임시휴점] 계절밥상':
                store[0] = '계절밥상'
            
            if store[0] not in total_store_list:
                total_store_list.append(store[0])
            
            else:
                continue
            
            dict_1 = {}
            dict_1["tag"] = store[0]
            dict_1["patterns"] = [store[0]]
            dict_1["responses"] = [store[0]+'<br>'+'위치: '+store[1] +'층'+'<br>'+'대표 전화: '+store[2]+'\n'] 
            for ment in 멘트:
                dict_1['patterns'].append(store[0]+' '+ment)
            tag_dicts.append(dict_1)
total_intents = {}
total_intents["intents"] = tag_dicts

In [350]:
import json
file_path = './intents_total_store.json'

with open(file_path, 'w', encoding='utf-8') as json_data:
    json.dump(total_intents, json_data, ensure_ascii=False, indent="\t")

In [24]:
intro_intents =  [
    {"tag": "greeting",
         "patterns": ["안녕", "안녕하세요", "잘 지냈어요?", "여기요", "잘 계셨어요?", "거기 누구 있어요?", "계세요", "계셔요", "좋은 날 입니다"],
         "responses": ["안녕하세요, 방문해 주셔서 감사합니다.", "다시 만나서 반가워요.", "만나서 반갑습니다", "찾아 주셔서 감사합니다", "안녕하세요, 어떻게 도와 드릴까요?"],
         "context_set": ""
        },
        {"tag": "goodbye",
         "patterns": ["안녕히 계셔요.", "그럼 나중에 또 들릴께요", "나중에요", "다음에", "다음번에", "장사 잘 하세요", "돈 많이 버세요", "수고하세요", "친절한 서비스에 감사합니다."],
         "responses": ["나중에 또 오세요, 방문해 주셔서 감사합니다.", "좋은 하루 되세요.", "안녕히 가세요", "감사합니다.", "고맙습니다", "즐거운 시간 되세요"]
        },
        {"tag": "thanks",
         "patterns": ["감사", "감사합니다", "고맙습니다", "좋은 서비스 감사합니다"],
         "responses": ["도움이 되서 다행입니다.", "당연히 할 일을 한건데요 뭐..", "저도 고맙습니다.", "감사합니다"]
        },
        {"tag": "Slang",
         "patterns": ["멍청이", "바보", "야임마", "꺼져", "사라져", "새끼", "닥쳐"],
         "responses": ["좋은 말로 해주세요.", "듣기가 거북하네요", "앞으로 더 노력 하겠습니다.", "좋은 모습으로 만나 뵐 수 있기를 바랍니다."]
        }
]


parking_intents =[
    		{
			"tag": "주차 요금",
			"patterns": [
			   "주차요금이 어떻게 되요?",
			   "주차비 알려주세요",
			   "승용차 주차요금 얼마에요?",
			   "대형버스 주차요금 얼마에요?",
			   "화물차 주차요금 얼마에요?",
			   "장애인 주차 요금 얼마에요??",
			   "국가유공자 주차요금 얼마에요?",
			   "국가 유공자 주차 할인 알려주세요",
			   "장애인 주차 할인 알려주세요"
			],
			"responses": [
			   "승용차 :20분 이내 출차시 무료 / 15분당 1,200원 / 1일 주차시 최고 48,000원<br>화물차(2.5t 이상) :승용차 주차요금의 2배<br>대형버스 :스타필드 코엑스몰 주차장 이용불가(탄천주차장 이용)<br>Grace Time :입차 후 20분 이내 출차 차량 – 주차료 면제<br>입차 후 20분 초과 출차 차량 – 정상요금(20분 포함)<br>무인요금계산기 정산 후 출차시 - Grace Time 20분 인정<br>장애인·국가유공자 :50% 할인"
			]
		 },
		 {
			"tag": "주차 할인",
			"patterns": [
			   "주차 할인되나요?",
			   "주차 할인 어떻게 되요?",
			   "주차비 얼마 할인되나요?",
			   "주차 할인 기준 알려줘요",
			   "주차 할인 얼마나 되나요?",
			   "주차 할인 적용 알려주세요",
			   "주차비 할인 적용 알려주세요"
			],
			"responses": [
			   "주요매장 적용기준 : 스타필드 코엑스몰 영수증 할인, 주요매장 주차할인, 주차할인권은 모두 중복가능<br>주요매장 할인기준 : 스타필드 코엑스몰 영수증 금액기준<br>일부 서비스/편의 매장은 제외(일부 매장별 자체할인 적용)<br>입차 후 구매금액에 한하며, 출차 시 자동정산 후에는 적용되지 않습니다.<br>일부 매장 별 자체할인 적용<br>5만원 이상 : 1시간 주차요금 무료<br>10만원 이상 : 2시간 주차요금 무료<br>15만원 이상 : 최대 3시간 주차요금 무료(3시간 이후 추가요금 발생)<br>주요매장 주차할인<br><br>메가박스(영화관)<br>관람고객<br>주간(06-22시) 최초 4시간까지 4,800원<br>야간(22-06시) 최초 4시간 2,400원 / 영화 상영 종료시간 기준<br>B2 매표소 PARKING 창구에서 차량번호로 할인인증]<br><br>아쿠아리움(수족관)<br>유료입장객 – 최초 4시간 4,800원<br>B1 매표소에서 차량번호로 할인인증"
			]
		 },
		 {
			"tag": "주차 진입로",
			"patterns": [
			   "주차장 어디로 들어가나요?",
			   "주차장 진입로가 어떻게 되나요?",
			   "주차 진입로 알려주세요",
			   "주차 북측 진입로 알려주세요",
			   "주차 서측 진입로 알려주세요"
			],
			"responses": [
			   "북측 진입로 : 지하 주차장 진입로<br>서측 진입로 : 지하 및 옥상 주차장 진입로"
			]
		 }
]